In [2]:
import psycopg2

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="telecom", 
    user="postgres", 
    password="123456", 
    host="localhost", 
    port="5432"
)

In [20]:
def header_data():
    curr = conn.cursor()
    curr.execute("SELECT * FROM xdr_data LIMIT 10;")
    rows = curr.fetchall()
    for row in rows:
        print('\t'.join(map(str, row)))
header_data()

1.31144834608449e+19	4/4/2019 12:01	770.0	4/25/2019 14:35	662.0	1823652.0	208201448079117.0	33664962239.0	35521209507511.0	9.16456699548519E+015	42.0	5.0	23.0	44.0	None	None	100.0	0.0	0.0	0.0	100.0	0.0	0.0	0.0	None	None	37624.0	38787.0	1823652892.0	Samsung	Samsung Galaxy A5 Sm-A520F	None	None	None	None	None	None	213.0	214.0	1545765.0	24420.0	1634479.0	1271433.0	3563542.0	137762.0	15854611.0	2501332.0	8198936.0	9656251.0	278082303.0	14344150.0	171744450.0	8814393.0	36749741.0	308879636.0
1.31144834828789e+19	4/9/2019 13:04	235.0	4/25/2019 8:15	606.0	1365104.0	208201909211140.0	33681854413.0	35794009006359.0	L77566A	65.0	5.0	16.0	26.0	None	None	100.0	0.0	0.0	0.0	100.0	0.0	0.0	0.0	None	None	168.0	3560.0	1365104371.0	Samsung	Samsung Galaxy J5 (Sm-J530)	None	None	None	None	None	None	971.0	1022.0	1926113.0	7165.0	3493924.0	920172.0	629046.0	308339.0	20247395.0	19111729.0	18338413.0	17227132.0	608750074.0	1170709.0	526904238.0	15055145.0	53800391.0	653384965.0
1.31144834840805e+19	4/9/2019 17

In [37]:
def get_top_10_handsets():
    try:
        # Create a cursor to execute queries
        cur = conn.cursor()
        
        # Execute the corrected query to get the top 10 handsets
        cur.execute("""
            SELECT "Handset Type", COUNT(*) AS usage_count
            FROM xdr_data
            GROUP BY "Handset Type"
            ORDER BY usage_count DESC
            LIMIT 10;
        """)
        
        # Fetch all rows from the result
        top_handsets = cur.fetchall()
        
        # Close the cursor
        cur.close()
        
        return top_handsets

    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Rollback in case of error
        return []  # Return an empty list in case of error

# Call the function and print the top 10 handsets
handsets = get_top_10_handsets()
if handsets:  # Check if handsets is not empty
    for handset, count in handsets:
        print(f"Handset: {handset}, Usage Count: {count}")
else:
    print("No handsets data available or an error occurred.")


Handset: Huawei B528S-23A, Usage Count: 19752
Handset: Apple iPhone 6S (A1688), Usage Count: 9419
Handset: Apple iPhone 6 (A1586), Usage Count: 9023
Handset: undefined, Usage Count: 8987
Handset: Apple iPhone 7 (A1778), Usage Count: 6326
Handset: Apple iPhone Se (A1723), Usage Count: 5187
Handset: Apple iPhone 8 (A1905), Usage Count: 4993
Handset: Apple iPhone Xr (A2105), Usage Count: 4568
Handset: Samsung Galaxy S8 (Sm-G950F), Usage Count: 4520
Handset: Apple iPhone X (A1901), Usage Count: 3813


In [41]:
def get_top_10_handsets():
    try:
        # Create a cursor to execute queries
        cur = conn.cursor()
        
        # Execute the corrected query to get the top 10 handsets
        cur.execute("""
            SELECT "Handset Manufacturer", COUNT(*) AS usage_count
            FROM xdr_data
            GROUP BY "Handset Manufacturer"
            ORDER BY usage_count DESC
            LIMIT 3;
        """)
        
        # Fetch all rows from the result
        top_handsets = cur.fetchall()
        
        # Close the cursor
        cur.close()
        
        return top_handsets

    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Rollback in case of error
        return []  # Return an empty list in case of error

# Call the function and print the top 10 handsets
handsets = get_top_10_handsets()
if handsets:  # Check if handsets is not empty
    for handset, count in handsets:
        print(f"Handset manufacturer: {handset}, Usage Count: {count}")
else:
    print("No handsets data available or an error occurred.")


Handset manufacturer: Apple, Usage Count: 59565
Handset manufacturer: Samsung, Usage Count: 40839
Handset manufacturer: Huawei, Usage Count: 34423


In [43]:
def get_top_5_handsets_per_top_3_manufacturers():
    try:
        # Create a cursor to execute queries
        cur = conn.cursor()

        # Execute the SQL query
        cur.execute("""
            WITH top_manufacturers AS (
                SELECT "Handset Manufacturer", COUNT(*) AS usage_count
                FROM xdr_data
                GROUP BY "Handset Manufacturer"
                ORDER BY usage_count DESC
                LIMIT 3
            )
            SELECT "Handset Manufacturer", "Handset Type", COUNT(*) AS usage_count
            FROM xdr_data
            WHERE "Handset Manufacturer" IN (SELECT "Handset Manufacturer" FROM top_manufacturers)
            GROUP BY "Handset Manufacturer", "Handset Type"
            ORDER BY "Handset Manufacturer", usage_count DESC
            LIMIT 5;
        """)

        # Fetch all rows from the result
        top_handsets = cur.fetchall()

        # Close the cursor
        cur.close()

        return top_handsets

    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Rollback in case of error
        return []  # Return an empty list in case of error

# Call the function and print the result
handsets = get_top_5_handsets_per_top_3_manufacturers()
if handsets:  # Check if handsets is not empty
    for manufacturer, handset, count in handsets:
        print(f"Manufacturer: {manufacturer}, Handset: {handset}, Usage Count: {count}")
else:
    print("No handset data available or an error occurred.")


    

Manufacturer: Apple, Handset: Apple iPhone 6S (A1688), Usage Count: 9419
Manufacturer: Apple, Handset: Apple iPhone 6 (A1586), Usage Count: 9023
Manufacturer: Apple, Handset: Apple iPhone 7 (A1778), Usage Count: 6326
Manufacturer: Apple, Handset: Apple iPhone Se (A1723), Usage Count: 5187
Manufacturer: Apple, Handset: Apple iPhone 8 (A1905), Usage Count: 4993


In [58]:
import pandas as pd
def size():
    try:
        cur = conn.cursor()
        cur.execute("SELECT * FROM  xdr_data")
        data = cur.fetchall()
        column_names = [column[0] for column in cur.description]
        data_frame = pd.DataFrame(data, columns = column_names)
        return data_frame.size
    except Exception as e:
        print(f"error: {e}")
    
size()
    

8250055

In [63]:
import pandas as pd
def shape():
    try:
        cur = conn.cursor()
        cur.execute("SELECT * FROM  xdr_data")
        data = cur.fetchall()
        column_names = [column[0] for column in cur.description]
        data_frame = pd.DataFrame(data, columns = column_names)
        return data_frame.size
    except Exception as e:
        print(f"error:{e}")
shape()

8250055

In [64]:
import pandas as pd
def info():
    try:
        cur = conn.cursor()
        cur.execute("SELECT * FROM  xdr_data")
        data = cur.fetchall()
        column_names = [column[0] for column in cur.description]
        data_frame = pd.DataFrame(data, columns = column_names)
        return data_frame.info()
    except Exception as e:
        print(f"error:{e}")
info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [67]:
import pandas as pd
def statisticalInfor():
    try:
        cur = conn.cursor()
        cur.execute("SELECT * FROM  xdr_data")
        data = cur.fetchall()
        column_names = [column[0] for column in cur.description]
        data_frame = pd.DataFrame(data, columns = column_names)
        return data_frame.describe()
    except Exception as e:
        print(f"error:{e}")
statisticalInfor()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150000.000000,150000.000000,1.500000e+05,1.494310e+05,1.489350e+05,1.494290e+05,122172.000000,122189.000000,150000.000000,...,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500000e+05,1.500000e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,...,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.611834,288.097653,8.103762e+04,2.148809e+10,2.447443e+12,2.241637e+13,619.782739,84.793524,23971.878541,...,6.710569e+06,6.345423e+06,6.725218e+06,6.359490e+06,2.439675e+08,4.782700e+06,2.432050e+08,4.769004e+06,1.127639e+07,2.441429e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,...,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744050e+04,2.082014e+14,3.365130e+10,3.546071e+13,32.000000,2.000000,43.000000,...,5.833501e+06,5.517965e+06,5.777156e+06,5.475981e+06,2.104733e+08,4.128476e+06,2.101869e+08,4.145943e+06,3.322201e+07,2.431068e+08
50%,7.349883e+18,499.000000,500.000000,8.639900e+04,2.082015e+14,3.366371e+10,3.572201e+13,45.000000,5.000000,63.000000,...,1.161602e+07,1.101345e+07,1.164222e+07,1.099638e+07,4.234081e+08,8.291208e+06,4.218030e+08,8.267071e+06,4.114331e+07,4.558411e+08
75%,1.304243e+19,749.000000,750.000000,1.324302e+05,2.082018e+14,3.368349e+10,8.611970e+13,70.000000,15.000000,19710.750000,...,1.744852e+07,1.651556e+07,1.747048e+07,1.650727e+07,6.331742e+08,1.243162e+07,6.316918e+08,1.238415e+07,4.903424e+07,6.657055e+08
max,1.318654e+19,999.000000,999.000000,1.859336e+06,2.140743e+14,8.823971e+14,9.900120e+13,96923.000000,7120.000000,378160.000000,...,2.325910e+07,2.201196e+07,2.325919e+07,2.201196e+07,8.434419e+08,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08


In [71]:
import pandas as pd

def fetch_data():
    try:
        cur = conn.cursor()
        cur.execute("SELECT * FROM xdr_data;")
        data = cur.fetchall()
        column_names = [column[0] for column in cur.description]
        data_frame = pd.DataFrame(data, columns=column_names)
        return data_frame
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def quantiles(quantile_values):
    try:
        data_frame = fetch_data()
        
        if data_frame is not None:
            # Convert columns to numeric where possible
            numeric_data_frame = data_frame.apply(pd.to_numeric, errors='coerce')

            # Calculate quantiles only for numeric columns
            result = numeric_data_frame.quantile(quantile_values)
            print(f"Quantiles for {quantile_values}:")
            print(result)
        else:
            print("No data to process.")
    except Exception as e:
        print(f"Error: {e}")

quantiles([0.25, 0.5, 0.75])  # Calculate 25th, 50th (median), and 75th percentiles


Quantiles for [0.25, 0.5, 0.75]:
         Bearer Id  Start  Start ms  End  End ms  Dur. (ms)          IMSI  \
0.25  7.349883e+18    NaN     250.0  NaN   251.0   57440.50  2.082014e+14   
0.50  7.349883e+18    NaN     499.0  NaN   500.0   86399.00  2.082015e+14   
0.75  1.304243e+19    NaN     749.0  NaN   750.0  132430.25  2.082018e+14   

      MSISDN/Number          IMEI  Last Location Name  ...  \
0.25   3.365130e+10  3.546071e+13        9.164567e+15  ...   
0.50   3.366371e+10  3.572201e+13        9.164567e+15  ...   
0.75   3.368349e+10  8.611970e+13        9.164567e+15  ...   

      Youtube DL (Bytes)  Youtube UL (Bytes)  Netflix DL (Bytes)  \
0.25           5833501.0           5517965.0           5777156.0   
0.50          11616019.0          11013447.0          11642217.0   
0.75          17448518.0          16515562.0          17470478.0   

      Netflix UL (Bytes)  Gaming DL (Bytes)  Gaming UL (Bytes)  \
0.25           5475981.0        210473253.0          4128476.0   
0.50

In [76]:
# Import the function from the utils module
from utils import fetch_data

# Assuming you have a valid connection object `conn`
data_frame = fetch_data(conn)
print(data_frame.isnull().sum())
    

Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [97]:
import pandas as pd
from utils import fetch_data

# Fetch data
df = fetch_data(conn)

def remove_outliers_iqr(df):
    try:
        # Print numeric columns
        numeric_columns = df.select_dtypes(include=['number']).columns
        print("Numeric columns in DataFrame:")
        print(numeric_columns)
        
        # Create a copy of the DataFrame to avoid modifying the original
        cleaned_df = df.copy()
        
        for column in numeric_columns:
            # Compute Q1 (25th percentile) and Q3 (75th percentile) for the column
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            print(f"Column '{column}' - Q1: {Q1}, Q3: {Q3}, Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")

            # Remove outliers
            cleaned_df = cleaned_df[(cleaned_df[column] >= lower_bound) & (cleaned_df[column] <= upper_bound)]
        
        print("DataFrame after outlier removal:")
        print(cleaned_df.head())
        return cleaned_df

    except Exception as e:
        print(f"Error: {e}")
        return None

# Call the function and assign the cleaned DataFrame to a new variable
cleaned_df = remove_outliers_iqr(df)


Numeric columns in DataFrame:
Index(['Bearer Id', 'Start ms', 'End ms', 'Dur. (ms)', 'IMSI', 'MSISDN/Number',
       'IMEI', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)',
       'Avg Bearer TP UL (kbps)', 'TCP DL Retrans. Vol (Bytes)',
       'TCP UL Retrans. Vol (Bytes)', 'DL TP < 50 Kbps (%)',
       '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)',
       'DL TP > 1 Mbps (%)', 'UL TP < 10 Kbps (%)',
       '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)',
       'UL TP > 300 Kbps (%)', 'HTTP DL (Bytes)', 'HTTP UL (Bytes)',
       'Activity Duration DL (ms)', 'Activity Duration UL (ms)', 'Dur. (ms).1',
       'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',
       'Nb of sec with Vol DL < 6250B', 'Nb of sec with

In [98]:
import pandas as pd
from utils import fetch_data

# Fetch data
df = fetch_data(conn)

def check_invalid_values(df):
    try:
        # Check for NaN values
        nan_count = df.isna().sum().sum()
        print(f"Total NaN values in the DataFrame: {nan_count}")

        # Check for NaN values in each column
        nan_per_column = df.isna().sum()
        print("NaN values per column:")
        print(nan_per_column)

        # Check for Infinite values
        inf_count = df.isin([float('inf'), float('-inf')]).sum().sum()
        print(f"Total Infinite values in the DataFrame: {inf_count}")

        # Check for Infinite values in each column
        inf_per_column = df.isin([float('inf'), float('-inf')]).sum()
        print("Infinite values per column:")
        print(inf_per_column)

        # Check for Non-Numeric Values in Numeric Columns
        numeric_columns = df.select_dtypes(include=['number']).columns
        for column in numeric_columns:
            non_numeric_values = df[~df[column].apply(pd.to_numeric, errors='coerce').notna()][column]
            if not non_numeric_values.empty:
                print(f"Non-numeric values in column '{column}':")
                print(non_numeric_values)

    except Exception as e:
        print(f"Error: {e}")

# Call the function
check_invalid_values(df)



Total NaN values in the DataFrame: 1031392
NaN values per column:
Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP